# Ch9:Up and Running with TensorFlow

* [Creating Your First Granph and Running It in a Session](#Creating-Your-First-Granph-and-Running-It-in-a-Session)
* [Lifecycle of Node Value](#Lifecycle-of-Node-Value)
* [Linear Regression with TensorFlow](#Linear-Regression-with-TensorFlow)
* [Implementing Gradient Descent](#Implementing Gradient Descent)
	* [Manually Computing the Gradients](#Manually-Computing-the-Gradients)
	* [Using autodiff](#Using-autodiff)
	* [Using an Optimizer](#Using-an-Optimizer)
* [Feeding Data to the Training Algorithm](#Feeding-Data-to-the-Training-Algorithm)
* [Saving and Restoring Models](#Saving-and-Restoring-Models)
* [Visualizing the Graph and Training Curves Using TensorBoard](#Visualizing-the-Graph-and-Training-Curves-Using-TensorBoard)
* [Name Scopes](#Name-Scopes)
* [Modularity](#Modularity)
* [Sharing Variables](#Sharing-Variables)

## Creating Your First Granph and Running It in a Session

![Alt](./9_1.png)

In [2]:
import tensorflow as tf
x=tf.Variable(3,name="X")

y=tf.Variable(4,name="y")
f=x*x*y+y+2

In [3]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)


42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
print(result)

42


In [7]:
init=tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result=f.eval()
print(result)

42


## Lifecyle of a Node Value

In [11]:
w=tf.constant(3)
x=w+2
y=x+5
z=x*3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [16]:
with tf.Session() as sess:
    y_val,z_val=sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with TensorFlow

In [17]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")

XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value=theta.eval()
print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


## Implementing Gradient Descent

### Manually Computing the Gradients

In [36]:
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()
scaled_housing_data=std_scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]

In [65]:
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch %100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

Epoch 0 MSE= 8.2536
Epoch 100 MSE= 0.74209
Epoch 200 MSE= 0.601004
Epoch 300 MSE= 0.583378
Epoch 400 MSE= 0.572428
Epoch 500 MSE= 0.563737
Epoch 600 MSE= 0.556689
Epoch 700 MSE= 0.550949
Epoch 800 MSE= 0.546262
Epoch 900 MSE= 0.542426


### Using autodiff

In [66]:
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
gradients=tf.gradients(mse,[theta])[0]
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch %100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

Epoch 0 MSE= 10.6348
Epoch 100 MSE= 0.695306
Epoch 200 MSE= 0.530887
Epoch 300 MSE= 0.527008
Epoch 400 MSE= 0.52622
Epoch 500 MSE= 0.525692
Epoch 600 MSE= 0.525311
Epoch 700 MSE= 0.525037
Epoch 800 MSE= 0.524838
Epoch 900 MSE= 0.524695


### Using an Optimizer

In [135]:
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch %100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

Epoch 0 MSE= 6.15226
Epoch 100 MSE= 0.801498
Epoch 200 MSE= 0.687328
Epoch 300 MSE= 0.652995
Epoch 400 MSE= 0.627381
Epoch 500 MSE= 0.607236
Epoch 600 MSE= 0.591277
Epoch 700 MSE= 0.578573
Epoch 800 MSE= 0.568417
Epoch 900 MSE= 0.560264


## Feeding Data to the Training Algorithm

In [69]:
A=tf.placeholder(tf.float32,shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_val_1=B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2=B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

In [70]:
print(B_val_1)

[[ 6.  7.  8.]]


In [71]:
print(B_val_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [126]:
def fetch_batch(batch_size):
    shuffle_indices=np.random.permutation(len(scaled_housing_data_plus_bias))
    X_batch_indices=shuffle_indices[:batch_size]
    y_batch_indices=shuffle_indices[:batch_size]
    return scaled_housing_data_plus_bias[X_batch_indices],housing.target[y_batch_indices].reshape(-1,1)

n_epochs=1000
learning_rate=0.01
batch_size=100
n_batches=int(np.ceil(m/batch_size))

X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(batch_size)
            if epoch %100==0:
                #print("Epoch",epoch,"MSE=",mse.eval())
                pass
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta=theta.eval()

In [128]:
print(best_theta)

[[ 2.06855249]
 [ 0.7787146 ]
 [ 0.14920233]
 [-0.09454302]
 [ 0.13140813]
 [ 0.00723424]
 [-0.04081651]
 [-0.72569853]
 [-0.68681014]]


## Saving and Restoring Models
Nothing to code.

## Visualizing the Graph and Training Curves Using TensorBoard

In [164]:
from datetime import datetime

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

print(logdir) 

n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")

mse_summary=tf.summary.scalar('MSE',mse)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch %100==0:
            #print("Epoch",epoch,"MSE=",mse.eval())
            summary_str=mse_summary.eval()
            file_writer.add_summary(summary_str,epoch)
        sess.run(training_op)
    best_theta=theta.eval()

file_writer.close()

tf_logs/run-20180126022357/


In [162]:
tf.reset_default_graph()

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)
print(logdir)
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(np.random.randn(9,1),dtype=tf.float32,name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
   
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")

mse_summary=tf.summary.scalar('MSE',mse)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch %100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
            summary_str=mse_summary.eval()
            file_writer.add_summary(summary_str,epoch)
        sess.run(training_op)
    best_theta=theta.eval()

tf_logs/run-20180126021650/
Epoch 0 MSE= 15.5801
Epoch 100 MSE= 0.774741
Epoch 200 MSE= 0.560504
Epoch 300 MSE= 0.550073
Epoch 400 MSE= 0.545522
Epoch 500 MSE= 0.541912
Epoch 600 MSE= 0.538932
Epoch 700 MSE= 0.536462
Epoch 800 MSE= 0.534413
Epoch 900 MSE= 0.532713


In [157]:
print(best_theta)

[[ 2.06855249]
 [ 1.02475023]
 [ 0.18495777]
 [-0.57879466]
 [ 0.54037702]
 [ 0.01704878]
 [-0.04908734]
 [-0.23402473]
 [-0.22471087]]


## Name Scopes

In [165]:
from datetime import datetime

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

print(logdir) 

n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
y_pred=tf.matmul(X,theta,name="predictions")

with tf.name_scope("loss") as scope:
    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error),name="mse")


mse_summary=tf.summary.scalar('MSE',mse)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch %100==0:
            #print("Epoch",epoch,"MSE=",mse.eval())
            summary_str=mse_summary.eval()
            file_writer.add_summary(summary_str,epoch)
        sess.run(training_op)
    best_theta=theta.eval()

file_writer.close()

tf_logs/run-20180126022753/


## Modularity

In [191]:
tf.reset_default_graph()
def relu(X):
    w_shape=(int(X.get_shape()[1]),1)
    w=tf.Variable(tf.random_normal(w_shape),name="weights")
    b=tf.Variable(0.0,name="bias")
    z=tf.add(tf.matmul(X,w),b,name="z")
    return tf.maximum(z,0.,name="relu")

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

print(logdir)

n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus,name="output")

file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())


tf_logs/run-20180126030220/


In [192]:
tf.reset_default_graph()
def relu(X):
    with tf.name_scope("relu"):
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape),name="weights")
        b=tf.Variable(0.0,name="bias")
        z=tf.add(tf.matmul(X,w),b,name="z")
        return tf.maximum(z,0.,name="relu")

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

print(logdir)

n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus,name="output")

file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())


tf_logs/run-20180126030457/


## Sharing Variables

In [195]:
tf.reset_default_graph()
def relu(X):
    with tf.name_scope("relu"):
        with tf.variable_scope("relu",reuse=True):
            threshold=tf.get_variable("threshold")
            w_shape=(int(X.get_shape()[1]),1)
            w=tf.Variable(tf.random_normal(w_shape),name="weights")
            b=tf.Variable(0.0,name="bias")
            z=tf.add(tf.matmul(X,w),b,name="z")
            return tf.maximum(z,0.,name="max")

now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs"
logdir="{}/run-{}/".format(root_logdir,now)

print(logdir)

n_features=3
X=tf.placeholder(tf.float32,shape=(None,n_features),name="X")
with tf.variable_scope("relu"):
    threshold=tf.get_variable("threshold",shape=(),initializer=tf.constant_initializer(0.0))
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus,name="output")

file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())


tf_logs/run-20180126033040/
